In [10]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from resnet_model import old_ResNet18, new_ResNet18, init_params, new_ResNet34, new_ResNet50, new_ResNet101, new_ResNet152
from network_func import *
from optimizer import *
from training import *
from useful_functions import *
from CIFAR10_data_preparation import *

In [17]:
n=5
I=np.eye(n)
one=np.ones(n)
R=np.outer(one,one)/n
#A,C=di_ring(n=n)
#B=get_B(A=A,u=2*n,n=n)
A,B=R,R
show_row(A)
_,__=test_row(A)
print(f"{_:.4e}, {__:.4e}")
h_data,y_data,X_test,y_test=cifar10_prepare_node_5_hard_shuffled()

A的第二大特征值: 4.790584196710678e-25
A的beta: 1.387778780781446e-16
A的spectral gap: 0.9999999999999999
A的kappa: 1.0000000000000002
S_A是: 4.472135954999581 

1.0000e+00, 5.0000e+00


In [20]:
import pandas as pd

# 初始化生成器对象
generator = new_train_PullSum(
    n=n,
    A=A,
    B=B,
    model_class=new_ResNet18,
    seed_for_model=49,
    criterion_class=nn.CrossEntropyLoss,
    epochs=50,
    lr=8e-4,
    batch_size=20,
    X_train_data=h_data,
    y_train_data=y_data,
    X_test_data=X_test,
    y_test_data=y_test,
    #compute_accuracy=CA_Resnet18,
    show_graph=True,
    try_init=True
)

df_history = pd.DataFrame(columns=["epoch", "train_loss", "train_accuracy", "test_accuracy"])

# 使用for循环逐步迭代，每个epoch结束后更新
for epoch, (train_loss_history, train_accuracy_history, test_accuracy_history) in enumerate(generator, start=1):
    # 获取当前epoch的最新数据
    Sum_l2 = train_loss_history[-1]  # 获取当前 epoch 的训练损失
    Sum_a2 = test_accuracy_history[-1]  # 获取当前 epoch 的测试集准确率
    Sum_train_acc = train_accuracy_history[-1]  # 获取当前 epoch 的训练集准确率

    # 将当前 epoch 的数据创建为 DataFrame
    current_epoch_data = pd.DataFrame({
        "epoch": [epoch],
        "train_loss": [Sum_l2],
        "train_accuracy": [Sum_train_acc],
        "test_accuracy": [Sum_a2]
    })

    # 使用 pd.concat 合并 DataFrame
    df_history = pd.concat([df_history, current_epoch_data], ignore_index=True)

    # 打印当前 epoch 的结果
    #print(f"Epoch {epoch} 结束: 训练损失 = {Sum_l2}, 训练集准确率 = {Sum_train_acc}, 测试集准确率 = {Sum_a2}")

    # 每个 epoch 后保存 DataFrame 到本地
    df_history.to_csv("/root/GanLuo/PullSum_MNIST/code/神经网络实验/CIFAR10最终实验/数据保存/ICLR/实验6,Resnet18,全连接,batch_zise=20,8e-4.csv", index=False)

Training Progress:   0%|          | 0/50 [01:57<?, ?it/s, epoch=1, loss=0.3559266461, test_accuracy=51.7700000000%, trian_accuracy=50.7860000000%]/tmp/ipykernel_43349/2166351716.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_history = pd.concat([df_history, current_epoch_data], ignore_index=True)
Training Progress:  50%|█████     | 25/50 [50:47<50:47, 121.88s/it, epoch=25, loss=0.0001514650, test_accuracy=77.9100000000%, trian_accuracy=99.9840000000%]  


KeyboardInterrupt: 